# Проект по анализу книжной БД для создания приложения

**Цель исследования** - предоставить данные, которые помогут сформулировать ценностное предложение для нового продукта: создаваемого приложения для тех, кто любит читать.

**Задачи исследования:**
Проанализировать базу данных и определить:
- сколько книг вышло после 1 января 2000 года;
- посчитать количество обзоров и среднюю оценку для каждой книги;
- определить издательство, которое выпустило наибольшее число книг толще 50-ти страниц;
- определить автора с самой высокой средней оценкой книг (среди книг с 50-ю и более оценками);
- посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Резюме

Выгрузка данных позволила узнать, что:
1. После 01.01.2000 года было выпущено 819 книг.
2. Издательство Penguin Books выпустило больще всего книг, толщиной более 50 страниц, а именно: 42 уникальных произведения.
3. Среди книг с 50-ю и более оценками, автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré с рейтингом 4.3.
4. Среднее количество обзоров от пользователей, поставивших более 50 оценок, равно 24,33.

Кроме того, для каждой книги, сведения о которой содержатся в БД, было определено количество обзоров и средний рейтинг.

## Загрузка файлов и знакомство с данными

### Импорт библиотек

Загружаю библиотеки, которые понадобятся мне для исследования:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Подключение к БД

Настраиваю параметры подключения к БД, в которой хранятся необходимые для исследования таблицы:

In [2]:
# устанавливаю параметры
db_config = {'user': 'user', # имя пользователя
'pwd': 'password', # пароль
'host': 'host',
'port': 1111, # порт подключения
'db': 'database_name'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняю коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

 Ошибок нет, значит подключение прошло успешно. Теперь выгружаю данные из БД:

### Загрузка таблицы `books`

In [3]:
pd.io.sql.read_sql('SELECT * FROM books LIMIT 5', con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Загрузка таблицы `authors`

In [4]:
pd.io.sql.read_sql('SELECT * FROM authors LIMIT 5', con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Загрузка таблицы `publisher`

In [5]:
pd.io.sql.read_sql('SELECT * FROM publishers LIMIT 5', con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Загрузка таблицы `ratings`

In [6]:
pd.io.sql.read_sql('SELECT * FROM ratings LIMIT 5', con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Загрузка таблицы `reviews`

In [7]:
pd.io.sql.read_sql('SELECT * FROM reviews LIMIT 5', con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Определение количества выпущенных книг

Необходимо посчитать количество книг, вышедших после 1 января 2000 года.

In [8]:
january_release = '''SELECT COUNT(book_id)
                  FROM books
                  WHERE publication_date > '2000-01-01'
                  '''

In [9]:
pd.io.sql.read_sql(january_release, con = engine) 

,count
0,819


После 01.01.2000 года было выпущено 819 книг

## Анализ количества обзоров и средних оценок книг

Необходимо посчитать количество обзоров и среднюю оценку каждой книги.

In [10]:
reviews_cnt = ''' WITH

                  rev AS
                  (SELECT book_id, COUNT(review_id) OVER (PARTITION BY book_id) AS cnt_rev
                  FROM reviews),
                  
                  rat AS 
                  (SELECT book_id, AVG(rating) OVER (PARTITION BY book_id) AS avg_rat
                  FROM ratings) 
                  
                  SELECT DISTINCT(b.book_id), title, cnt_rev, avg_rat
                  FROM books AS b
                  LEFT JOIN rev ON rev.book_id = b.book_id
                  LEFT JOIN rat ON rat.book_id = b.book_id
                  ORDER BY 1
                  
             '''

In [11]:
pd.io.sql.read_sql(reviews_cnt, con = engine)

,book_id,title,cnt_rev,avg_rat
0,1,'Salem's Lot,2.0,3.666667
1,2,1 000 Places to See Before You Die,1.0,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.666667
3,4,1491: New Revelations of the Americas Before C...,2.0,4.500000
4,5,1776,4.0,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.666667
996,997,Xenocide (Ender's Saga #3),3.0,3.400000
997,998,Year of Wonders,4.0,3.200000
998,999,You Suck (A Love Story #2),2.0,4.500000


Для каждой книги, сведения о которой содержатся в БД, определено количество обзоров и средний рейтинг.

## Определение топ-издательств по выпуску книг

Необходимо определите издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [12]:
publishing = ''' SELECT publisher,
                        COUNT (book_id) OVER (PARTITION BY publisher_id)
                 FROM (SELECT b.publisher_id, publisher, book_id, num_pages
                      FROM books AS b
                      LEFT JOIN publishers AS pub ON pub.publisher_id = b.publisher_id) AS pub_table
                 WHERE num_pages > 50
                 ORDER BY 2 DESC
                 LIMIT 1
             '''

In [13]:
pd.io.sql.read_sql(publishing, con = engine) 

,publisher,count
0,Penguin Books,42


Издательство Penguin Books выпустило больще всего книг, толщиной более 50 страниц, а именно: 42 уникальных произведения.

## Определение самого популярного автора

Необходимо определить автора с самой высокой средней оценкой книг (среди книг с 50-ю и более оценками).

In [14]:
author_top_review = ''' WITH

                        rev_50 AS
                        (SELECT *
                        FROM (SELECT *,
                              COUNT(rating) OVER (PARTITION BY book_id) AS review_cnt
                              FROM (SELECT b.book_id, a.author_id, a.author, r.rating
                                     FROM books AS b
                                     LEFT JOIN authors AS a ON b.author_id = a.author_id
                                     LEFT JOIN ratings AS r ON r.book_id = b.book_id) AS book_reviews) AS book_rev_cnt
                        WHERE review_cnt > 50),
                        
                        book_rev_avg AS
                        (SELECT *,
                               AVG(rating) OVER (PARTITION BY book_id)AS avg_review_book
                        FROM rev_50)
                        
                        SELECT DISTINCT (author_id), author,
                               AVG(avg_review_book) OVER (PARTITION BY author_id) AS avg_review_author
                        FROM book_rev_avg
                        ORDER BY 3 DESC
                        LIMIT 1
                    '''

In [15]:
pd.io.sql.read_sql(author_top_review, con = engine) 

,author_id,author,avg_review_author
0,236,J.K. Rowling/Mary GrandPré,4.287097


Среди книг с 50-ю и более оценками, автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré с рейтингом 4.3

## Определени количества обзоров среди наиболее активных пользователей

Необходимо определить среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [16]:
avg_cnt_rev = ''' SELECT AVG(count)
                  FROM (SELECT DISTINCT username, COUNT(review_id) OVER (PARTITION BY username)
                  FROM reviews
                  WHERE username IN (SELECT username
                                    FROM (SELECT username,
                                          COUNT(rating)
                                    FROM ratings
                                    GROUP BY 1) AS rat_cnt
                                    WHERE count > 50)) AS rev_cnt
              '''

In [17]:
pd.io.sql.read_sql(avg_cnt_rev, con = engine) 

,avg
0,24.333333


Среднее количество обзоров от пользователей, поставивших более 50 оценок, равно 24,33.